# Preprocess Input Data and Create processed Json file

이 노트북은 주어진 파일에서 중복된 내용을 제거하고, 영어 데이터를 한국어로 번역하여 추후에 사용하려고 함니다.

---

## [중요] 사전 실행 노트북
이 노트북은 아래 두개의 셋업 노트북이 먼저 실행이 되어야 합니다.
- (1) Setup 노트북
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup.ipynb 와 같습니다.
    -  [Setup Notebook](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup.ipynb)


# 1. Set up

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)


python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot']


# 2. Remvoe duplicated contents 

In [2]:
from local_utils.proc_docs import get_load_json, show_doc_json

In [3]:
import glob
import pandas as pd

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/*.json'
json_files = glob.glob(folder_path)

def remove_duplicate_col(json_files, remove_col):
    '''
    json_files : list of json file path
    remove_col : column name to remove duplicate
    Desc: 
    '''
    all_df = pd.DataFrame()
    
    for file_path in json_files:
        df = pd.read_json(file_path)
        df = pd.json_normalize(df.sections) # Normalize sections
        all_df = pd.concat([all_df, df])
    
    all_df.drop_duplicates(subset=remove_col, inplace=True)
    
    return all_df

df = remove_duplicate_col(json_files, remove_col='content')
df

,title,last_updated,content,url,project
0,How-to videos,2023-09-27,How-to videos. Contains videos on how to use K...,https://docs.samsungknox.com/admin/efota-one/h...,EFOTA
1,Knox E-FOTA,2023-07-26,Knox E-FOTA. Knox E-FOTA enables enterprise IT...,https://docs.samsungknox.com/admin/efota-one,EFOTA
2,EMM compatibility matrix,2023-07-26,EMM compatibility matrix. The following table ...,https://docs.samsungknox.com/admin/efota-one/a...,EFOTA
3,Install an app through an EMM,2023-07-26,Install an app through an EMM. Explains how to...,https://docs.samsungknox.com/admin/efota-one/a...,EFOTA
4,About EMM groups,2023-07-26,About EMM groups. Describes how Knox E-FOTA al...,https://docs.samsungknox.com/admin/efota-one/e...,EFOTA
...,...,...,...,...,...
71,"What is my customer ID, and where can I find it?",2023-07-26,"What is my customer ID, and where can I find i...",https://docs.samsungknox.com/admin/care-plus/k...,SCPE
72,What is a KDP reseller?,2023-07-26,What is a KDP reseller?. Overview The term _KD...,https://docs.samsungknox.com/admin/care-plus/k...,SCPE
73,What is the 'license assignment'?,2023-07-26,What is the 'license assignment'?. Overview A ...,https://docs.samsungknox.com/admin/care-plus/k...,SCPE
74,What is SWAP and BER?,2023-07-26,What is SWAP and BER?. Overview SWAP - A new d...,https://docs.samsungknox.com/admin/care-plus/k...,SCPE


# 3. Craete new json file

In [4]:
cleaned_json_file_path = "data/poc/preprocessed_json/all_processed_data.json"
df.reset_index(drop=True).to_json(cleaned_json_file_path,orient='records')

Verify the created json file

In [5]:
df_json = pd.read_json(cleaned_json_file_path)
df_json.head(3)

,title,last_updated,content,url,project
0,How-to videos,2023-09-27,How-to videos. Contains videos on how to use K...,https://docs.samsungknox.com/admin/efota-one/h...,EFOTA
1,Knox E-FOTA,2023-07-26,Knox E-FOTA. Knox E-FOTA enables enterprise IT...,https://docs.samsungknox.com/admin/efota-one,EFOTA
2,EMM compatibility matrix,2023-07-26,EMM compatibility matrix. The following table ...,https://docs.samsungknox.com/admin/efota-one/a...,EFOTA


In [6]:
%store cleaned_json_file_path

Stored 'cleaned_json_file_path' (str)


# 4. Translate English to Korean

In [7]:
import boto3
translate_client = boto3.client(service_name='translate', region_name='us-west-2', use_ssl=True)

## Text Splitter 정의 
- AWS Translate 는 최대 10000 bytes 를 번역할 수 있어서, 조금 더 작은 단위로 문서를 쪼개서 번역 함

In [8]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, SpacyTextSplitter

chunk_size = 8192
chunk_overlap = 0

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

In [11]:
%%time

import pandas as pd

def translate_df(df, target_col, new_col):
    
    def run_translate(text):
        '''
        Split doc into small chunks, translate them, and then merge into one
        '''
        doc = [Document(page_content=text)]
        docs = text_splitter.split_documents(doc)                
        
        translate_doc = []
        for i, chunk in enumerate(docs):
            # print("English Text: \n", chunk.page_content)                    
            result = translate_client.translate_text(Text=chunk.page_content, 
                    SourceLanguageCode="en", TargetLanguageCode="ko")
            result = result['TranslatedText']
            translate_doc.append(result)
        # print("# of chunks: ", i)
            
        translate_doc = ' '.join(translate_doc)            
        # print("translate_doc: \n", translate_doc)
        
        return translate_doc
    
    def translate(df):
        text = df[target_col]
        result = run_translate(text)
        # print(result)        

        return result
        
    # Start here
    df[new_col] = df.apply(translate, axis=1)
    
    return df




CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 6.68 µs


## 번역 실행

In [12]:
%%time
df_kr = translate_df(df, target_col='title', new_col='title_ko')


CPU times: user 2.43 s, sys: 110 ms, total: 2.54 s
Wall time: 2min 17s


In [14]:
df_kr = translate_df(df_kr, target_col='content', new_col='content_ko')

In [17]:
df_kr.head()

,title,last_updated,content,url,project,title_ko,content_ko
0,How-to videos,2023-09-27,How-to videos. Contains videos on how to use K...,https://docs.samsungknox.com/admin/efota-one/h...,EFOTA,사용 방법 동영상,사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있...
1,Knox E-FOTA,2023-07-26,Knox E-FOTA. Knox E-FOTA enables enterprise IT...,https://docs.samsungknox.com/admin/efota-one,EFOTA,녹스 E-FOTA,녹스 E-FOTA.Knox E-FOTA를 사용하면 기업 IT 관리자가 사용자 개입 ...
2,EMM compatibility matrix,2023-07-26,EMM compatibility matrix. The following table ...,https://docs.samsungknox.com/admin/efota-one/a...,EFOTA,EMM 호환성 매트릭스,EMM 호환성 매트릭스.다음 표는 다음을 수행할 수 있는 다양한 EMM의 기능을 보...
3,Install an app through an EMM,2023-07-26,Install an app through an EMM. Explains how to...,https://docs.samsungknox.com/admin/efota-one/a...,EFOTA,EMM을 통해 앱 설치,EMM을 통해 앱을 설치합니다.EMM을 통해 앱을 설치하는 방법을 설명합니다.이 섹...
4,About EMM groups,2023-07-26,About EMM groups. Describes how Knox E-FOTA al...,https://docs.samsungknox.com/admin/efota-one/e...,EFOTA,EMM 그룹에 대한 정보,EMM 그룹에 대해.Knox E-FOTA를 사용하여 EMM에서 디바이스 그룹을 동기...


In [18]:
def post_process_translate(ldf):
    df = ldf.copy()
    df.drop(columns=['title', 'content'], inplace=True)
    df.rename(columns={'title_ko': 'title', 'content_ko': 'content'}, inplace=True)

    return df

df_post = post_process_translate(df_kr)
df_post

,last_updated,url,project,title,content
0,2023-09-27,https://docs.samsungknox.com/admin/efota-one/h...,EFOTA,사용 방법 동영상,사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있...
1,2023-07-26,https://docs.samsungknox.com/admin/efota-one,EFOTA,녹스 E-FOTA,녹스 E-FOTA.Knox E-FOTA를 사용하면 기업 IT 관리자가 사용자 개입 ...
2,2023-07-26,https://docs.samsungknox.com/admin/efota-one/a...,EFOTA,EMM 호환성 매트릭스,EMM 호환성 매트릭스.다음 표는 다음을 수행할 수 있는 다양한 EMM의 기능을 보...
3,2023-07-26,https://docs.samsungknox.com/admin/efota-one/a...,EFOTA,EMM을 통해 앱 설치,EMM을 통해 앱을 설치합니다.EMM을 통해 앱을 설치하는 방법을 설명합니다.이 섹...
4,2023-07-26,https://docs.samsungknox.com/admin/efota-one/e...,EFOTA,EMM 그룹에 대한 정보,EMM 그룹에 대해.Knox E-FOTA를 사용하여 EMM에서 디바이스 그룹을 동기...
...,...,...,...,...,...
71,2023-07-26,https://docs.samsungknox.com/admin/care-plus/k...,SCPE,내 고객 ID는 무엇이며 어디서 찾을 수 있나요?,내 고객 ID는 무엇이며 어디서 찾을 수 있나요?.개요 Knox Admin Port...
72,2023-07-26,https://docs.samsungknox.com/admin/care-plus/k...,SCPE,KDP 리셀러란 무엇인가요?,KDP 리셀러란 무엇입니까?.개요 _KDP_라는 용어는 Knox 배포 프로그램의 약...
73,2023-07-26,https://docs.samsungknox.com/admin/care-plus/k...,SCPE,'라이선스 할당'이란 무엇입니까?,'라이선스 할당'이란 무엇입니까?.개요 라이선스 자체에는 어떤 서비스를 위한 것인지...
74,2023-07-26,https://docs.samsungknox.com/admin/care-plus/k...,SCPE,스왑과 BER이란 무엇입니까?,스왑과 BER이란 무엇입니까?개요 스왑 - 새 디바이스가 고객에게 제공되고 기존 디...


# 5. Craete new korean Json file

In [19]:
def create_json_file(df, file_name):
    df.to_json(file_name, orient='records', force_ascii=False)
    print(f"{file_name} is generated")

cleaned_json_file_path = "data/poc/preprocessed_json/all_processed_data_ko.json"    
create_json_file(df_post, file_name="data/poc/preprocessed_json/all_processed_data_ko.json")


data/poc/preprocessed_json/all_processed_data_ko.json is generated


In [20]:
df_json = pd.read_json(cleaned_json_file_path)
df_json.head(3)

,last_updated,url,project,title,content
0,2023-09-27,https://docs.samsungknox.com/admin/efota-one/h...,EFOTA,사용 방법 동영상,사용 방법 동영상.Knox E-FOTA를 사용하는 방법에 대한 비디오가 포함되어 있...
1,2023-07-26,https://docs.samsungknox.com/admin/efota-one,EFOTA,녹스 E-FOTA,녹스 E-FOTA.Knox E-FOTA를 사용하면 기업 IT 관리자가 사용자 개입 ...
2,2023-07-26,https://docs.samsungknox.com/admin/efota-one/a...,EFOTA,EMM 호환성 매트릭스,EMM 호환성 매트릭스.다음 표는 다음을 수행할 수 있는 다양한 EMM의 기능을 보...
